In [1]:
import numpy as np
import os
import zipfile
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
print(len(os.listdir("C:/Users/adeat/Desktop/Project/Class/Tensorflow/Hist_OCT/Train/Hist")))
print(len(os.listdir("C:/Users/adeat/Desktop/Project/Class/Tensorflow/Hist_OCT/Train/oct")))
print(len(os.listdir("C:/Users/adeat/Desktop/Project/Class/Tensorflow/Hist_OCT/validate/Hist")))
print(len(os.listdir("C:/Users/adeat/Desktop/Project/Class/Tensorflow/Hist_OCT/validate/OCT")))

1577
206
535
206


In [3]:
model = tf.keras.models.Sequential([
            tf.keras.layers.Conv2D(32, (3,3), activation ='relu', input_shape=(150,150,3)),
            tf.keras.layers.MaxPooling2D(2,2),
            #tf.keras.layers.Conv2D(32, (3,3), activation ='relu'),
            #tf.keras.layers.MaxPooling2D(2,2),
            tf.keras.layers.Conv2D(64, (3,3), activation ='relu'),
            tf.keras.layers.MaxPooling2D(2,2),
            tf.keras.layers.Conv2D(64, (3,3), activation ='relu'),
            tf.keras.layers.MaxPooling2D(2,2),
            tf.keras.layers.Flatten(),
            tf.keras.layers.Dense(512, activation = tf.nn.relu),
            tf.keras.layers.Dense(1, activation= 'sigmoid')
])

In [4]:
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [5]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 148, 148, 32)      896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 74, 74, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 72, 72, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 36, 36, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 34, 34, 64)        36928     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 17, 17, 64)        0         
_________________________________________________________________
flatten (Flatten)            (None, 18496)             0

In [6]:
train_datagen = ImageDataGenerator(rescale=1/255.0)
validation_datagen = ImageDataGenerator(rescale=1/255.0)

training_generator = train_datagen.flow_from_directory(
                    "C:/Users/adeat/Desktop/Project/Class/Tensorflow/Hist_OCT/Train",
                    target_size = (150, 150),
                    batch_size = 100,
                    class_mode = 'binary')

validation_generator = validation_datagen.flow_from_directory(
                    "C:/Users/adeat/Desktop/Project/Class/Tensorflow/Hist_OCT/validate",
                    target_size = (150, 150),
                    batch_size = 50,
                    class_mode = 'binary')

Found 1783 images belonging to 2 classes.
Found 741 images belonging to 2 classes.


In [7]:
model.fit_generator(
        training_generator,
        steps_per_epoch = 10,
        epochs = 5,
        validation_data = validation_generator,
        validation_steps = 10,
        verbose = 2)

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/5
10/10 - 295s - loss: 0.2009 - accuracy: 0.8680 - val_loss: 0.0272 - val_accuracy: 0.9920
Epoch 2/5
10/10 - 142s - loss: 0.0059 - accuracy: 0.9980 - val_loss: 0.0048 - val_accuracy: 0.9980
Epoch 3/5
10/10 - 74s - loss: 0.0058 - accuracy: 0.9990 - val_loss: 0.0050 - val_accuracy: 0.9960
Epoch 4/5
10/10 - 43s - loss: 1.7518e-04 - accuracy: 1.0000 - val_loss: 7.3431e-05 - val_accuracy: 1.0000
Epoch 5/5
10/10 - 42s - loss: 5.5045e-05 - accuracy: 1.0000 - val_loss: 1.5517e-05 - val_accuracy: 1.0000


In [8]:
model.evaluate(validation_generator)

15/15 [==============================] - 6s 395ms/step - loss: 1.9136e-05 - accuracy: 1.0000


[1.913563028210774e-05, 1.0]

In [27]:
from os import listdir
from os.path import isfile, join
predict_dir_path='C:/Users/adeat/Desktop/Project/Class/Tensorflow/Hist_OCT/validate/test/'
onlyfiles = [f for f in listdir(predict_dir_path) if isfile(join(predict_dir_path, f))]
print(onlyfiles)

['HP_BC_001.jpg', 'HP_BC_002.jpg', 'HP_BC_004.jpg', 'OCT_BC_001.jpg', 'OCT_BC_002.jpg', 'OCT_BC_004.jpg']


In [29]:

# predicting images
from keras.preprocessing import image
OCT_counter = 0 
Hist_counter  = 0
for file in onlyfiles:
    img = image.load_img(predict_dir_path+file, target_size=(150, 150))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    
    images = np.vstack([x])
    classes = model.predict_classes(images, batch_size=10)
    classes = classes[0][0]
    
    if classes == 0:
        print(file + ": " + 'Histopathology')
        Hist_counter += 1
    else:
        print(file + ": " + 'OCT')
        OCT_counter += 1
print("Total OCT  :",OCT_counter)
print("Total histopathology :",Hist_counter)

HP_BC_001.jpg: Histopathology
HP_BC_002.jpg: Histopathology
HP_BC_004.jpg: Histopathology
OCT_BC_001.jpg: OCT
OCT_BC_002.jpg: OCT
OCT_BC_004.jpg: OCT
Total OCT  : 3
Total histopathology : 3
